In [ ]:
try:
    from altinn3 import *
    from skatteetaten_api import main_relay, base64_decode_response, decode_dokument
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from pathlib import Path
except ImportError as e:
    print("Mangler en eller avhengighet, installer dem via pip, se requierments.txt fil for detaljer")
    raise ImportError(e)


In [ ]:
def hent_gjeldende_skattemelding_og_sjekk_foraandsfastsettingstype(orgnr, inntektsaar):
    url_gjeldende = f'{base_url}/api/skattemelding/v2/{inntektsaar}/{orgnr}'
    r_gjeldende = s.get(url_gjeldende)

    if r_gjeldende:
        sme_og_naering_respons = xmltodict.parse(r_gjeldende.text)
        skattemelding_base64 = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]["skattemeldingdokument"]
        decoded_sme_xml = decode_dokument(skattemelding_base64)
        sme_xml = xml.dom.minidom.parseString(decoded_sme_xml["content"]).toprettyxml()
        sme_dict = xmltodict.parse(sme_xml)
        if "gjelderForhaandsfastsetting" in sme_dict["skattemelding"]:
            forhaandsfastsetting =  sme_dict["skattemelding"]["gjelderForhaandsfastsetting"]['innsendingsformat']['forhaandsfastsettingsformattype']
        else:
            forhaandsfastsetting = "Parten har publisert skattemelding, men ikke klargjort for foråndsfastsetting, vil følge ordniært fastsettings løp"
        return forhaandsfastsetting
    else:
        return (r_gjeldende.status_code, r_gjeldende.text)


In [ ]:
inntektsaar = "2022"
base_url = "https://idporten-api-sbstest.sits.no"
orgnr = "313816079" # bruk TENOR til å finne en part uten fastsetting av skattemeldingen og hvem som er daglig leder for orgnr (15859797943)


In [ ]:
idporten_header = main_relay()
s = requests.Session()
s.headers = dict(idporten_header)


In [ ]:
#Hent gjeldende skattemelding, sjekker om den er klar, og om den har forhåndsfastsetting informasjon
sme_status = hent_gjeldende_skattemelding_og_sjekk_foraandsfastsettingstype(orgnr, inntektsaar)
print(f"Orgnr har følgende status: {sme_status} ")


In [ ]:
url_klargjoer_forhaandsfastsetting = f"{base_url}/api/skattemelding/v2/klargjoerforhaandsfastsetting/{inntektsaar}/{orgnr}"


idporten_header["content-type"] = "application/json"
r_klargjoer = requests.post(url_klargjoer_forhaandsfastsetting, headers=idporten_header)
if r_klargjoer and ("application/json" in r_klargjoer.headers["content-type"]): 
    print(r_klargjoer.text)
else:
    print(r_klargjoer)
    

In [ ]:
url_gjeldende = f'{base_url}/api/skattemelding/v2/{inntektsaar}/{orgnr}'
r_gjeldende = s.get(url_gjeldende)

if r_gjeldende:

    sme_og_naering_respons = xmltodict.parse(r_gjeldende.text)
    skattemelding_base64 = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"]["skattemeldingdokument"]
    decoded_sme_xml = decode_dokument(skattemelding_base64)
    sme_xml = xml.dom.minidom.parseString(decoded_sme_xml["content"]).toprettyxml()
    sme_dict = xmltodict.parse(sme_xml)
    if "gjelderForhaandsfastsetting" in sme_dict["skattemelding"]:
        forhaandsfastsetting = sme_dict["skattemelding"]["gjelderForhaandsfastsetting"]
    else:
        forhaandsfastsetting = False
    print(f"Skattemeldingen er publisert, og foråhndsfastsettings type: {forhaandsfastsetting['innsendingsformat']['forhaandsfastsettingsformattype']} ")


else:
    print(r_gjeldende)